In [80]:
useMock=False

In [81]:
from cffi import FFI
ffi = FFI()
with open('extracted_headers/safe_app_datatype_declarations','r') as f:
    myHeaders=f.read()
ffi.cdef(myHeaders)
with open('extracted_headers/safe_app_function_declarations','r') as f:
    funs=f.read()
ffi.cdef(funs)
lib=ffi.dlopen('../compiled_binaries/libsafe_app' + useMock*'_mock' + '.so')
libAuth=ffi.dlopen('../compiled_binaries/libsafe_authenticator' + useMock*'_mock' + '.so')

---

In [82]:
from threading import Thread

def threader(function):
    def innerThread(*args):
        oneThread=Thread(target=function,args=tuple(args))
        oneThread.start()
    return innerThread

In [83]:
from IPython.core import magic_arguments
from IPython.core.magic import line_magic, cell_magic, line_cell_magic, Magics, magics_class
 
@magics_class
class TestMagics(Magics):
    @cell_magic
    def threadcell(self, line='', cell=None):
        @threader
        def threadedAction():
            exec(cell)
        threadedAction()
 
    @line_magic
    def thread(self, line):
        @threader
        def threadedAction():
            exec(line)
        threadedAction()
 
ip = get_ipython()
ip.register_magics(TestMagics)

In [84]:
import time

In [85]:
def print3():
    for i in range(3):
        print(i)
        time.sleep(1)

%thread print3()
%thread print3()

%%threadcell
print3()

%%threadcell
print3()

---

maybe it makes most sense to really define the data types by hand and just check them for changes with updates on the api - not entirely sure

- rid

---

```c
typedef unsigned long int uintptr_t;

typedef struct {
 char* id;
 char* scope;
 char* name;
 char* vendor;
} AppExchangeInfo;
```

In [86]:
class AppExchangeInfo:
    def __init__(self,id=b'noId',scope=b'noScope',name=b'noName',vendor=b'nobody'):
        self.id = ffi.new('char[]',id)
        self.scope = ffi.new('char[]',scope)
        self.name = ffi.new('char[]',name)
        self.vendor = ffi.new('char[]',vendor)

        self.entity = ffi.new('AppExchangeInfo *',[self.id,self.scope,self.name,self.vendor])

In [87]:
def AppExchangeInfo(id=b'noId',scope=b'noScope',name=b'noName',vendor=b'nobody'):
    id = ffi.new('char[]',id)
    scope = ffi.new('char[]',scope)
    name = ffi.new('char[]',name)
    vendor = ffi.new('char[]',vendor)

    myStruct = ffi.new('AppExchangeInfo *',[id,scope,name,vendor])
    
    return myStruct, [id, scope, name, vendor]

newExChangeInfo,infopayload = AppExchangeInfo(b'0.0.2',name=b'pyTest',vendor=b'rid')

newExChangeInfo,infopayload = AppExchangeInfo(id=b'meh',scope=b'myScope',
                                              name=b'PyTests only today',vendor=b'rid')

ffi.string(newExChangeInfo.vendor)

ffi.string(newExChangeInfo.name)

```c
typedef struct {
 _Bool read;
 _Bool insert;
 _Bool update;
 _Bool delete;
 _Bool manage_permissions;
} PermissionSet;
```

In [88]:
def PermissionSet(read=True,insert=True,update=True,delete=True,manage_permissions=True):
    return ffi.new('PermissionSet *',[read,insert,update,delete,manage_permissions])

newPermissionSet = PermissionSet()

```c
typedef struct {
 char* cont_name;
 PermissionSet access;
} ContainerPermissions;
```

In [89]:
def ContainerPermissions(name=b'noName',access=None):
    containerName = ffi.new('char[]',name)
    if not access:
        access = PermissionSet()
    container = ffi.new('ContainerPermissions *',[containerName,access[0]])
    
    return container, [containerName,access]

newPermissions,permission_addon=ContainerPermissions(b'_pictures',newPermissionSet)

In [90]:
newPermissions,permission_addon=ContainerPermissions(b'_publicNames')

In [91]:
newPermissions,permission_addon=ContainerPermissions(b'_music')

In [92]:
ffi.string(newPermissions.cont_name)

b'_music'

```c
typedef struct {
 AppExchangeInfo app;
 _Bool app_container;
 ContainerPermissions* containers;
 uintptr_t containers_len;
 uintptr_t containers_cap;
} AuthReq;
```

In [93]:
def AuthReq(permissions,containers_len,containers_cap,id=b'noId',scope=b'pythonscript',
            name=b'noName',vendor=b'nobody',app_container=True):
    
    newExChangeInfo,infopayload = AppExchangeInfo(id,scope,name,vendor)
    
    authReq = ffi.new('AuthReq *',[newExChangeInfo[0],app_container,permissions,containers_len,containers_cap])
    
    return authReq, [newExChangeInfo,infopayload]

myAuth,addData=AuthReq(newPermissions,0,0,id=b'ridsTest',scope=b'pythonscript'
                       ,name=b'PyTest',vendor=b'rid never work',app_container=True)

myAuth,addData=AuthReq(newPermissions,0,0,id=b'ridsTest',scope=b'pythonscript'
                       ,name=b'PyTest',vendor=b'nobody',app_container=True)

In [94]:
myAuth,addData=AuthReq(newPermissions,1,1,id=b'newProgId_unique_thing',scope=b'lalali'
                       ,name=b'newProgramName',vendor=b'nobody',app_container=True)

In [97]:
@ffi.callback("void(void *, FfiResult *)")
def feedback(userData, result):
    print(userData)
    print(result.error_code)

In [98]:
lib.app_set_additional_search_path(ffi.new('char[]',b'../compiled_binaries/'),ffi.NULL,feedback)

<cdata 'void *' NULL>
0


@ffi.callback("void(void *, FfiResult *, unsigned int, char *)")
def o_cb(userData, result, noClueWhatNumber, resultString):
    print(userData)
    print(result)
    print(noClueWhatNumber)
    print(resultString)

lib.encode_auth_req(myAuth,ffi.NULL,o_cb)

myAuth.containers.access.insert=True

myAuth.containers.access.delete=True

myAuth.containers.access.read=True

In [18]:
ffi.string(myAuth.app.name)

b'newProgramName'

In [19]:
ffi.string(myAuth.app.id)

b'newProgId_unique_thing'

In [20]:
encodedAuth=None

In [21]:
@ffi.callback("void(void *, FfiResult *, unsigned long int, char *)")
def o_cb(userData, result, reqId, encodedReq):
    global encodedAuth
    print(userData)
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(reqId)
    print(ffi.string(encodedReq))
    encodedAuth = ffi.string(encodedReq)

In [22]:
@ffi.callback("void(void *, FfiResult *, unsigned int, char *)")
def o_cb(userData, result, reqId, encodedReq):
    global encodedAuth
    print(userData)
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(reqId)
    print(ffi.string(encodedReq))
    encodedAuth = ffi.string(encodedReq)

In [23]:
lib.encode_auth_req(myAuth,ffi.NULL,o_cb)

<cdata 'void *' NULL>
0
1200104540
b'bAAAAAAC4ESEEOAAAAAABMAAAAAAAAAAANZSXOUDSN5TUSZC7OVXGS4LVMVPXI2DJNZTQCBQAAAAAAAAAABWGC3DBNRUQ4AAAAAAAAAAANZSXOUDSN5TXEYLNJZQW2ZIGAAAAAAAAAAAG433CN5SHSAIBAAAAAAAAAAAAMAAAAAAAAAAAL5WXK43JMMCQAAAAAAAAAAAAAAAAAAIAAAAAEAAAAABQAAAAAQAAAAAA'


In [24]:
b'safe-auth://' + encodedAuth

b'safe-auth://bAAAAAAC4ESEEOAAAAAABMAAAAAAAAAAANZSXOUDSN5TUSZC7OVXGS4LVMVPXI2DJNZTQCBQAAAAAAAAAABWGC3DBNRUQ4AAAAAAAAAAANZSXOUDSN5TXEYLNJZQW2ZIGAAAAAAAAAAAG433CN5SHSAIBAAAAAAAAAAAAMAAAAAAAAAAAL5WXK43JMMCQAAAAAAAAAAAAAAAAAAIAAAAAEAAAAABQAAAAAQAAAAAA'

In [25]:
import base64

In [26]:
base64.encodebytes(ffi.string(myAuth.app.id))

b'bmV3UHJvZ0lkX3VuaXF1ZV90aGluZw==\n'

In [27]:
base64.b64encode(b'newProgId_unique_thing').strip(b'=')

b'bmV3UHJvZ0lkX3VuaXF1ZV90aGluZw'

In [28]:
schemeName=b'safe-'+base64.b64encode(b'newProgId_unique_thing').strip(b'=')

In [29]:
schemeName

b'safe-bmV3UHJvZ0lkX3VuaXF1ZV90aGluZw'

In [30]:

sysUri = FFI()
with open('extracted_headers/safe_app_datatype_declarations','r') as f:
    myHeaders=f.read()
sysUri.cdef(myHeaders)
with open('extracted_headers/system_uri.h','r') as f:
    funs=f.read()
sysUri.cdef(funs)
lib2=sysUri.dlopen('../compiled_binaries/libsystem_uri.so')

In [31]:
@sysUri.callback("void(void *, FfiResult *)")
def installerFeedback(userData, result):
    print(userData)
    print(result.error_code)

```c
void install(char* bundle, 
             char* vendor, 
             char* name, 
             char* exec_args, 
             uint_least64_t exec_args_len, 
             char* icon, 
             char* schemes, 
             void* user_data, 
             void (*o_cb)(void* user_data, FfiResult* result));
```

In [32]:
print(ffi.string(myAuth.app.name))
print(ffi.string(myAuth.app.vendor))
print(ffi.string(myAuth.app.scope))

b'newProgramName'
b'nobody'
b'lalali'


In [33]:
import os
print(os.getcwd())

/home/riddim/safe/pySafe/safenet


In [34]:
pathToHandler=(os.getcwd()+'/uriHandler.py').encode()

In [35]:
pathToHandler

b'/home/riddim/safe/pySafe/safenet/uriHandler.py'

```c
myAuth,addData=AuthReq(newPermissions,1,1,id=b'newProgId',scope=b'lalali'
                       ,name=b'newProgramName',vendor=b'nobody',app_container=False)
```

In [36]:
bundle=sysUri.new('char[]',ffi.string(myAuth.app.id))
vendor=sysUri.new('char[]',ffi.string(myAuth.app.vendor))
name=sysUri.new('char[]',ffi.string(myAuth.app.name))
exec_args=sysUri.new('char[]',b'python')
exec_args2=sysUri.new('char[]',pathToHandler)
exec_args_len=2
icon=sysUri.new('char[]',b'')
schemes=sysUri.new('char[]',schemeName)
user_data=sysUri.NULL

In [37]:
exec_args_1 = sysUri.new('char*[]',[exec_args,exec_args2])

In [38]:
lib2.install(bundle,vendor,name,exec_args_1 ,exec_args_len,icon,schemes,user_data,installerFeedback)

<cdata 'void *' NULL>
0


```c 
void open_uri(char* uri, 
              void* user_data, 
              void (*o_cb)(void* user_data, FfiResult* result));
```

In [39]:
@sysUri.callback("void(void *, FfiResult *)")
def openerFeedback(userData, result):
    global res
    res = result
    print(userData)
    print(result.error_code)

In [40]:
ffi.string(myAuth.app.name)

b'newProgramName'

In [41]:
ffi.string(myAuth.app.vendor)

b'nobody'

In [42]:
ffi.string(myAuth.app.name)

b'newProgramName'

In [43]:
uri=sysUri.new('char[]',b'safe-auth://' + encodedAuth)
user_data=sysUri.NULL

In [44]:
lib2.open_uri(uri,user_data,openerFeedback)

<cdata 'void *' NULL>
0


In [45]:
def printCrustConfig(configAddress,configlength=42):
    lali=[]
    for i in range(configlength):
        lali.append(configAddress[i])
    print(lali)

```c 
void decode_ipc_msg(char* msg, 
                    void* user_data, 
                    void (*o_auth)(void* user_data, uint32_t req_id, AuthGranted* auth_granted), 
                    void (*o_unregistered)(void* user_data, uint32_t req_id, uint8_t* serialised_cfg, 
                                            uintptr_t serialised_cfg_len), 
                    void (*o_containers)(void* user_data, uint32_t req_id), 
                    void (*o_share_mdata)(void* user_data, uint32_t req_id), 
                    void (*o_revoked)(void* user_data), 
                    void (*o_err)(void* user_data, FfiResult* result, uint32_t req_id));
```

In [46]:
import time

In [47]:
import safeUtils

In [48]:
@ffi.callback("void(void *, uint32_t, AuthGranted *)")
def o_auth_decode_ipc_msg(userData, req_id, auth_granted):
    
    printCrustConfig(auth_granted.bootstrap_config)
    
    print('o_auth')
    print(userData)
    print(req_id)
    returnDict = ffi.from_handle(userData)
    #returnDict['AuthGranted'] = picData
    returnDict['AuthGranted'] = safeUtils.copy(auth_granted,ffi)
    #print(returnDict)
    myContainer=auth_granted.access_container_entry.containers[0]
    print(myContainer)
    print(auth_granted.access_container_entry.containers_len)
    print(auth_granted.access_container_entry.containers_cap)
    print(ffi.string(myContainer.name))
    print(myContainer.mdata_info)
    print(ffi.sizeof(myContainer.mdata_info))
    print(myContainer.permissions)
    print(ffi.sizeof(myContainer.permissions))


In [49]:
@ffi.callback("void(void *, uint32_t, uint8_t *, uintptr_t)")
def o_unregistered_decode_ipc_msg(userData, req_id, serialised_cfg, serialised_cfg_len):
    print('o_unregistered')
    print(req_id)

In [50]:
@ffi.callback("void(void *, uint32_t)")
def o_containers_decode_ipc_msg(userData, req_id):
    print('o_containers')
    print(req_id)

In [51]:
@ffi.callback("void(void *, uint32_t)")
def o_share_mdata_decode_ipc_msg(userData, req_id):
    print('o_share')
    print(req_id)

In [52]:
@ffi.callback("void(void *)")
def o_revoked_decode_ipc_msg(userData):
    print('o_revoked')
    print(req_id)

In [53]:
@ffi.callback("void(void *, FfiResult *, uint32_t)")
def o_error_decode_ipc_msg(userData, result, req_id):
    global res
    res = result
    print('o_err')
    print(userData)
    print(req_id)
    print(result.error_code)
    print(ffi.string(result.description))

In [54]:
with open('answer','r') as f:
    answer=f.readlines()

In [55]:
import re
data=re.compile("'safe-.+?:(.+?)'")

In [56]:
my=data.findall(answer[-1])

In [57]:
msg = ffi.new('char[]',my[-1].encode())

In [58]:
my[-1]

'bAEAAAAC4ESEEOAAAAAAAAAAAAAQAAAAAAAAAAACGXDI7FJQJGFWTNW3C6KKWWNKBOOU4OVE3T23X7XQZHHMYK6SFOMQAAAAAAAAAAAFGOIG3WJ3KMXRCCSNVNM3OYESI5S5VJZJHUW2AKWHM24FUXZAUMQQAAAAAAAAAAAFMS46SBKZB7SAXDL7TK6RHYR5MV6PUEO3IULWJPPTGX4WC75CTUFAAAAAAAAAAAAEPIUOBVE2QSQTAG3FRNKOW7CWMHWQAGJNO45CKT4B7WJUFUARH6CWJOPJAVMQ7ZALRV7ZVPIT4I6WK7H2CHNUKF3EXXZTL6LBP6RJ2CIAAAAAAAAAAAAAIAFRVA65NQ2WBY65WGIG3ZNAYDHDGWFQEPF5BCB26CS7I2Z5UUIAAAAAAAAAAACBYAYDI7XWL6FTSOGLDWXWLICA7A4OUUKJF72STQDPUMZ7EUFZPAGIAAAAAAAAAAAIAAAAAAAAAAABRG44C4NRSFY3TMLRYHI2TIOBTCMAAAAAAAAAAAMJTHAXDMOBOGE4DKLRSGE4DUNJUHAZREAAAAAAAAAAAGEZTQLRWHAXDCOBRFY2TOORVGQ4DGEQAAAAAAAAAAAYTGOBOGY4C4MJYGEXDMMB2GU2DQMYSAAAAAAAAAAADCMZYFY3DQLRRHAYS4OBWHI2TIOBTCIAAAAAAAAAAAMJTHAXDMOBOGE4DCLRYG45DKNBYGMJQAAAAAAAAAABRGM4C4NRYFYYTQMJOGE3DQORVGQ4DGEYAAAAAAAAAAAYTGOBOGY4C4MJYGEXDCNZWHI2TIOBTCMAAAAAAAAAAAMJTHAXDMOBOGE4DCLRRG44TUNJUHAZRGAAAAAAAAAAAGEZTQLRWHAXDCOBRFYYTQMB2GU2DQMYTAAAAAAAAAAADCMZYFY3DQLRRHAYS4MJYGI5DKNBYGMJQAAAAAAAAAABRGM4C4NRYFYYTQMJOGI2DEORVGQ4DGEYAAAAAAAAAAAYTGO

In [59]:
returnDict={}

In [60]:
myHand=ffi.new_handle(returnDict)

lib.decode_ipc_msg(msg,myHand,o_auth_decode_ipc_msg,o_unregistered_decode_ipc_msg
                   ,o_containers_decode_ipc_msg,o_share_mdata_decode_ipc_msg,
                   o_revoked_decode_ipc_msg,o_error_decode_ipc_msg)

In [61]:
%%threadcell
lib.decode_ipc_msg(msg,myHand,o_auth_decode_ipc_msg,o_unregistered_decode_ipc_msg
                   ,o_containers_decode_ipc_msg,o_share_mdata_decode_ipc_msg,
                   o_revoked_decode_ipc_msg,o_error_decode_ipc_msg)

[25, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0, 49, 55, 56, 46, 54, 50, 46, 55, 54, 46, 56, 58, 53, 52, 56, 51, 19, 0, 0, 0, 0, 0, 0, 0, 49, 51]
o_auth
<cdata 'void *' 0x7f3e406f4958>
1200104540
<cdata 'ContainerInfo &' 0x7f3e20004980>
2
2
b'_music'
<cdata 'MDataInfo &' 0x7f3e20004988>
160
<cdata 'PermissionSet &' 0x7f3e20004a28>
5


#time.sleep(3)
lib.decode_ipc_msg(msg,ffi.new_handle(returnDict),o_auth_decode_ipc_msg,o_unregistered_decode_ipc_msg
                   ,o_containers_decode_ipc_msg,o_share_mdata_decode_ipc_msg,
                   o_revoked_decode_ipc_msg,o_error_decode_ipc_msg)

----

now the app

ffi.cdef('''
typedef struct { ...; } foo_t;
''')

In [62]:
@ffi.callback("void(void*)")
def disconnect_app_registered(user_data):
    print('disconnected - callback from app_registered')

In [63]:
innerData=None
@ffi.callback("void(void* , FfiResult* , App* app)")
def result_app_registered(user_data, result, app):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    returnDict = ffi.from_handle(user_data)
    returnDict['app']=app

lib.app_registered(myAuth.app.id,grantedAuth,ffi.NULL,disconnect_app_registered,result_app_registered)

In [64]:
%%threadcell

lib.app_registered(myAuth.app.id,returnDict['AuthGranted'][0],ffi.new_handle(returnDict),disconnect_app_registered,result_app_registered)

@ffi.callback("void(void* , FfiResult*)")
def result_clearCache(user_data, result):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))

lib.app_reset_object_cache(returnDict['app'],ffi.new_handle(returnDict),result_clearCache)

In [68]:
with open('myNewMutableData_as_bytes','rb') as f:
    readData=f.read()

In [69]:
firstPythonDataOnSafe=ffi.new('MDataInfo *')

In [70]:
ffi.buffer(firstPythonDataOnSafe)[:]=readData

```c
pub unsafe extern "C" fn mdata_list_keys(
    app: *const App, 
    info: *const FfiMDataInfo, 
    user_data: *mut c_void, 
    o_cb: extern "C" fn(_: *mut c_void, _: *const FfiResult, _: *const FfiMDataKey, _: usize)
)
```

In [77]:
@ffi.callback("void(void* , FfiResult*, MDataKey*, uint64_t)")
def result_mdata_list_keys(user_data, result, key, size):
    #print('results')
    #print('error code: ')
    #print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    #print(key.val_len)
    #print(size)
    if size>0:
        #print(key.val_len)
        print(ffi.string(key.key))
    #returnDict = ffi.from_handle(user_data)
    #returnDict['myMdata']=mdataInfo

In [78]:
lib.mdata_list_keys(returnDict['app'],firstPythonDataOnSafe,ffi.NULL,result_mdata_list_keys)

b'dask'


In [79]:
lib.mdata_list_values(returnDict['app'],firstPythonDataOnSafe,ffi.NULL,result_mdata_list_values)

b'we get closer...'


```c
pub unsafe extern "C" fn mdata_list_values(
    app: *const App, 
    info: *const FfiMDataInfo, 
    user_data: *mut c_void, 
    o_cb: extern "C" fn(_: *mut c_void, _: *const FfiResult, _: *const FfiMDataValue, _: usize)
)
```

In [72]:
@ffi.callback("void(void* , FfiResult*, MDataValue*, uint64_t)")
def result_mdata_list_values(user_data, result, value, size):
    #print('results')
    #print('error code: ')
    #print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    #print(key.val_len)
    #print(size)
    if size>0:
        #print(value.content_len)
        print(ffi.string(value.content))
    #returnDict = ffi.from_handle(user_data)
    #returnDict['myMdata']=mdataInfo

In [73]:
lib.mdata_list_values(returnDict['app'],firstPythonDataOnSafe,ffi.NULL,result_mdata_list_values)

NameError: name 'newMdata' is not defined

In [67]:
containerName=ffi.new('unsigned char[]',b'_music')

In [68]:
@ffi.callback("void(void* , FfiResult*, MDataInfo *)")
def result_mdataInfoContainer(user_data, result, mdataInfo):
    global myMdata
    myMdata=mdataInfo
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(mdataInfo)
    #returnDict = ffi.from_handle(user_data)
    #returnDict['myMdata']=mdataInfo

In [69]:
lib.access_container_get_container_mdata_info(returnDict['app'],containerName,ffi.new_handle(returnDict),result_mdataInfoContainer)

results
error code: 
0
<cdata 'MDataInfo *' 0x7f67c02f3238>


In [70]:
returnDict['newMdata']=myMdata

In [71]:
myMdata.type_tag

15000

```c
pub unsafe extern "C" fn mdata_list_keys(
    app: *const App, 
    info: *const FfiMDataInfo, 
    user_data: *mut c_void, 
    o_cb: extern "C" fn(_: *mut c_void, _: *const FfiResult, _: *const FfiMDataKey, _: usize)
)
```

In [72]:
@ffi.callback("void(void* , FfiResult*, MDataKey*, uint64_t)")
def result_mdata_list_keys(user_data, result, key, size):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    #print(key.val_len)
    print(size)
    if size>0:
        print(key.val_len)
        print(ffi.string(key.val))
    #returnDict = ffi.from_handle(user_data)
    #returnDict['myMdata']=mdataInfo

In [73]:
lib.mdata_list_keys(returnDict['app'],myMdata,ffi.NULL,result_mdata_list_keys)

results
error code: 
0
0


In [253]:
lib.mdata_list_keys(returnDict['app'],returnDict['random'],ffi.NULL,result_mdata_list_keys)

results
error code: 
0
1
5
b'dask'


In [71]:
lib.mdata_list_keys(returnDict['app'],newMdata,ffi.NULL,result_mdata_list_keys)

results
error code: 
0
1
5
b'dask'


In [94]:
with open('myNewMutableData_as_bytes_3','wb') as f:
    writeData=ffi.buffer(returnDict['random'])[:]
    f.write(writeData)

In [65]:
with open('myNewMutableData_as_bytes','rb') as f:
    readData2=f.read()

In [66]:
readData2

b'\x8f\x7f\xdd\x83\x1e\x0e\xf3^\xb7\xf4ie\xb8\xb0\x91^ce"\xa0\xff\xda\x8cs&\x19\x83\xb5\x01\x88(\x9d\t\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x85R\xb2U\x00\x00'

In [202]:
returnDict['random']

<cdata 'MDataInfo *' owning 160 bytes>

In [209]:
myMdata.type_tag

140594294999616

In [254]:
dataBuffer=ffi.buffer(returnDict['random'])

In [255]:
dataBuffer[:]

b'\x8f\x7f\xdd\x83\x1e\x0e\xf3^\xb7\xf4ie\xb8\xb0\x91^ce"\xa0\xff\xda\x8cs&\x19\x83\xb5\x01\x88(\x9d\t\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x85R\xb2U\x00\x00'

In [200]:
copyData=ffi.from_buffer(dataBuffer)

In [216]:
ffi.string(copyData)

b'\xc0\xa4\x7fR\xb2U'

In [67]:
newMdata=ffi.new('MDataInfo *')

In [203]:
newMdata.type_tag

0

In [211]:
insertBuffer=ffi.buffer(newMdata)

In [236]:
ffi.buffer(myMdata)[:]

b'\xc0\xa4\x7fR\xb2U\x00\x00X\xd3\xb1j\xde\x7f\x00\x00H\x90\x94j\xde\x7f\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00@\xca\x02\xa9\xde\x7f\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\xa1\x85R\xb2U\x00\x00H\x90\x94j\xde\x7f\x00\x00\x81C\x81R\xb2U\x00\x00@\xca\x02\xa9\xde\x7f\x00\x00\x00|\xb0\xb0\x02Y\xae\x8d@\xca\x02\xa9\xde\x7f\x00\x00\xb8\xe0\x9c\xa3\xde\x7f\x00\x00\xb8\xe0\x9c\xa3\xde\x7f\x00\x00\x9a\x9f\x8aR\xb2U\x00\x00\x10\xcf\x9cj\xde\x7f\x00\x00\x00|\xb0\xb0\x02Y\xae\x8d\x00\xab\xb0j\xde\x7f\x00\x00\x00\xab\xb0j\xde\x7f\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00'

In [223]:
ffi.sizeof(copyData)

160

In [224]:
freshData=ffi.unpack(copyData,ffi.sizeof(copyData))

In [239]:
ffi.buffer(newMdata)[:]=ffi.buffer(returnDict['random'])[:]

In [68]:
ffi.buffer(newMdata)[:]=readData2

In [69]:
newMdata.type_tag

777

In [180]:
myMdata.type_tag

140594294999616

In [249]:
copied=ffi.new('MDataInfo *')

In [250]:
ffi.buffer(copied)[:]=readData2

In [251]:
copied.type_tag

777

In [118]:
returnDict['random']

<cdata 'MDataInfo *' owning 160 bytes>

createMutableData

@ffi.callback("void(void* , FfiResult*, MDataInfo *)")
def result_mdataInfoRand(user_data, result, mdataInfo):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(mdataInfo)
    returnDict = ffi.from_handle(user_data)
    returnDict['newMdata']=ffi.new('MDataInfo *',mdataInfo[0])

lib.mdata_info_random_public(15150,ffi.new_handle(returnDict),result_mdataInfoRand)

lali=returnDict['newMdata']

for i in range(22):
    print(lali.name[i])

@ffi.callback("void(void* , FfiResult*, MDataInfo *)")
def result_mdataInfoRand(user_data, result, mdataInfo):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(mdataInfo)
    returnDict = ffi.from_handle(user_data)
    returnDict['newMdata_private']=ffi.new('MDataInfo *',mdataInfo[0])

lib.mdata_info_random_private(15000,ffi.new_handle(returnDict),result_mdataInfoRand)

permission

In [74]:
@ffi.callback("void(void* , FfiResult*, unsigned long)")
def result_newPermissions(user_data, result, handle):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(handle)

In [75]:
lib.mdata_permissions_new(returnDict['app'],ffi.NULL,result_newPermissions)

results
error code: 
0
1


In [76]:
@ffi.callback("void(void* , FfiResult* , unsigned long)")
def result_mdata_entries_new(user_data, result, entriesHandle):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    #noway = ffi.from_handle(user_data)
    print(entriesHandle)
    #print(noway)
    #noway['mdata'] = entriesHandle

In [77]:
lib.mdata_entries_new(returnDict['app'],ffi.NULL,result_mdata_entries_new)

results
error code: 
0
2


In [82]:
oneKey=ffi.new('unsigned char[]',b'safe for Python is coming')

In [83]:
len(oneKey)

26

In [78]:
value=ffi.new('unsigned char[]',b'and together we will change the world!!!! MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA ')

In [79]:
len(value)

8107

In [80]:
@ffi.callback("void(void* , FfiResult*)")
def result_mdata_entries_insert(user_data, result):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))

In [84]:
lib.mdata_entries_insert(returnDict['app'],2,oneKey,len(oneKey),value,len(value),ffi.NULL,result_mdata_entries_insert)

results
error code: 
0


In [85]:
ffi.string(oneKey)

b'safe for Python is coming'

In [86]:
@ffi.callback("void(void* , FfiResult*, unsigned char*, unsigned long, unsigned long)")
def result_mdata_entries_get(user_data, result, value, val_length, noClueWhatThisIs):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(ffi.string(value))
    print(noClueWhatThisIs)

In [87]:
lib.mdata_entries_get(returnDict['app'],2,oneKey,len(oneKey),ffi.NULL,result_mdata_entries_get)

results
error code: 
0
b'and together we will change the world!!!! MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHAHAAHAHAHAHA MUAAAHAHAHAHHAHAHAHAHAHAHHAHAHAHAHAHHAHAHAHAHAHHAHA

```c
pub unsafe extern "C" fn mdata_entry_actions_new(
    app: *const App, 
    user_data: *mut c_void, 
    o_cb: extern "C" fn(_: *mut c_void, _: *const FfiResult, _: MDataEntryActionsHandle)
)
```

In [101]:
@ffi.callback("void(void* , FfiResult*, unsigned long)")
def result_mdata_entry_actions_new(user_data, result, handle):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(handle)

In [149]:
lib.mdata_entry_actions_new(returnDict['app'],ffi.NULL,result_mdata_entry_actions_new)

results
error code: 
0
6


```c
pub unsafe extern "C" fn mdata_entry_actions_insert(
    app: *const App, 
    actions_h: MDataEntryActionsHandle, 
    key: *const u8, 
    key_len: usize, 
    value: *const u8, 
    value_len: usize, 
    user_data: *mut c_void, 
    o_cb: extern "C" fn(_: *mut c_void, _: *const FfiResult)
)
```

In [154]:
@ffi.callback("void(void* , FfiResult*)")
def result_mdata_entry_actions_insert(user_data, result):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    #print(handle)

In [155]:
lib.mdata_entry_actions_insert(returnDict['app'],6,oneKey,len(oneKey),value,len(value),ffi.NULL,result_mdata_entry_actions_insert)

results
error code: 
0


```c
pub unsafe extern "C" fn mdata_info_random_public(
    type_tag: u64, 
    user_data: *mut c_void, 
    o_cb: extern "C" fn(_: *mut c_void, _: *const FfiResult, _: *const FfiMDataInfo)
)
```

In [88]:
myMdata2=None

In [89]:
@ffi.callback("void(void* , FfiResult*, MDataInfo*)")
def result_mdata_info_random_public(user_data, result, mData):
    global myMdata2
    myMdata2=mData
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    returnDict = ffi.from_handle(user_data)
    returnDict['random']=ffi.new('MDataInfo*',mData[0])
    print(mData.type_tag)

In [90]:
lib.mdata_info_random_public(777,ffi.new_handle(returnDict),result_mdata_info_random_public)

results
error code: 
0
777


In [105]:
returnDict['random'].type_tag

777

In [106]:
myMdata

<cdata 'MDataInfo *' 0x7f723cc15238>

In [107]:
myMdata.type_tag

0

In [108]:
lib.access_container_get_container_mdata_info(returnDict['app'],containerName,ffi.new_handle(returnDict),result_mdataInfoContainer)

results
error code: 
0
<cdata 'MDataInfo *' 0x7f723cc15238>


In [88]:
myMdata.type_tag

15000

In [90]:
newMDataInfo.type_tag

140594294999616

In [91]:
newMDataInfo = ffi.new('MDataInfo*',myMdata[0])

In [93]:
newMDataInfo.type_tag

15000

In [88]:
lib.mdata_info_random_public(777,ffi.new_handle(returnDict),result_mdata_info_random_public)

results
error code: 
0
777


In [90]:
myMdata2.type_tag

0

In [91]:
myMdata.type_tag

0

In [112]:
returnDict['newData']

<cdata 'MDataInfo *' 0x7ffe209f5e20>

In [113]:
returnDict['newData'].type_tag

0

In [90]:
lib.access_container_get_container_mdata_info(returnDict['app'],containerName,ffi.new_handle(returnDict),result_mdataInfoContainer)

results
error code: 
0
<cdata 'MDataInfo *' 0x7f9598494238>


In [91]:
myMdata.type_tag

15000

In [92]:
myMdata

<cdata 'MDataInfo *' 0x7f9598494238>

```c
pub unsafe extern "C" fn mdata_put(
    app: *const App, 
    info: *const FfiMDataInfo, 
    permissions_h: MDataPermissionsHandle, 
    entries_h: MDataEntriesHandle, 
    user_data: *mut c_void, 
    o_cb: extern "C" fn(_: *mut c_void, _: *const FfiResult)
)
```

In [91]:
@ffi.callback("void(void* , FfiResult*)")
def result_mdata_put(user_data, result):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))

In [92]:
lib.mdata_put(returnDict['app'],returnDict['random'],1,2,ffi.NULL,result_mdata_put)

results
error code: 
0


In [93]:
returnDict['random'].type_tag

777

In [105]:
returnDict['newData'].type_tag

0

In [109]:
returnDict['newData'].enc_key[3]

0

```c
pub unsafe extern "C" fn mdata_mutate_entries(
    app: *const App, 
    info: *const FfiMDataInfo, 
    actions_h: MDataEntryActionsHandle, 
    user_data: *mut c_void, 
    o_cb: extern "C" fn(_: *mut c_void, _: *const FfiResult)
)
```

In [138]:
@ffi.callback("void(void* , FfiResult*)")
def result_mdata_mutate_entries(user_data, result):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))

In [100]:
returnDict['newMdata'].type_tag

94655951974784

In [136]:
returnDict['random'].type_tag

777

In [165]:
lib.mdata_mutate_entries(returnDict['app'],returnDict['random'],3,ffi.NULL,result_mdata_mutate_entries)

results
error code: 
0


In [166]:
lib.mdata_mutate_entries(returnDict['app'],returnDict['random'],6,ffi.NULL,result_mdata_mutate_entries)

results
error code: 
-100
b'Core error: Routing client error -> Access denied'


In [109]:
@ffi.callback("void(void* , FfiResult* , unsigned long)")
def result_app_pub_sign_key(user_data, result, entriesHandle):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(entriesHandle)

In [110]:
lib.app_pub_sign_key(returnDict['app'],ffi.NULL,result_app_pub_sign_key)

results
error code: 
0
3


In [111]:
@ffi.callback("void(void *, FfiResult *)")
def result_mdata_put(user_data, result):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))

In [112]:
lib.mdata_put(returnDict['app'],returnDict['newMdata'],1,2,ffi.NULL,result_mdata_put)

results
error code: 
0


In [71]:
newPermissions=PermissionSet()

In [164]:
newPermissions.access.read

True

In [73]:
@ffi.callback("void(void* , FfiResult*)")
def result_setPermissions(user_data, result):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    #returnDict = ffi.from_handle(user_data)
    #returnDict['newMdata']=ffi.new('PermissionSet *',handle[0])

In [74]:
lib.mdata_set_user_permissions(returnDict['app'],returnDict['newMdata_private'],
                               3,newPermissions,1,ffi.NULL,result_setPermissions)

results
error code: 
-100
b'Core error: Routing client error -> Access denied'


In [77]:
@ffi.callback("void(void* , FfiResult* , unsigned long, unsigned long)")
def result_sign_generate_key_pair(user_data, result, pubKeyHandle, secKeyHandle):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(pubKeyHandle)
    print(secKeyHandle)

In [78]:
lib.sign_generate_key_pair(returnDict['app'],ffi.NULL,result_sign_generate_key_pair)

results
error code: 
0
4
5


In [75]:
@ffi.callback("void(void* , FfiResult*, PermissionSet*)")
def result_getPermissions(user_data, result, handle):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(handle)
    #returnDict = ffi.from_handle(user_data)
    #returnDict['newMdata']=ffi.new('PermissionSet *',handle[0])

In [77]:
lib.mdata_permissions_get(returnDict['app'],1,4,ffi.new_handle(returnDict),result_getPermissions)

results
error code: 
-1011
b'Invalid sign public key handle'
<cdata 'PermissionSet *' NULL>


In [75]:
@ffi.callback("void(void *, FfiResult *)")
def result_mdata_put(user_data, result):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))

In [76]:
lib.mdata_put(returnDict['app'],returnDict['newMdata'],1,2,ffi.NULL,result_mdata_put)

results
error code: 
-100
b'Core error: Routing client error -> Access denied'


In [105]:
@ffi.callback("void(void* , FfiResult*, PermissionSet*)")
def result_getPermissions(user_data, result, handle):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(handle)
    returnDict = ffi.from_handle(user_data)
    returnDict['newMdata']=ffi.new('PermissionSet *',handle[0])

In [ ]:
lib.mdata_permissions_get(returnDict['app'],3,0,ffi.new_handle(returnDict),result_getPermissions)

results
error code: 
-1011
b'Invalid sign public key handle'
<cdata 'PermissionSet *' NULL>


permissionSet

In [62]:
@ffi.callback("void(void* , FfiResult*, unsigned long)")
def result_clearCache(user_data, result, handle):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(handle)

In [63]:
lib.mdata_permissions_new(returnDict['app'],ffi.NULL,result_clearCache)

results
error code: 
0
1


In [62]:
@ffi.callback("void(void* , FfiResult*, unsigned long)")
def result_clearCache(user_data, result, handle):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(handle)

In [63]:
lib.idata_new_self_encryptor(returnDict['app'],ffi.NULL,result_clearCache)

results
error code: 
0
1


In [64]:
with open('picData.jpeg','rb') as f:
    picData = f.read()

In [67]:
writeData=ffi.new('unsigned char[]',picData)

In [68]:
ffi.sizeof(writeData)

113025

In [69]:
@ffi.callback("void(void* , FfiResult*)")
def result_writeData(user_data, result):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))

In [71]:
lib.idata_write_to_self_encryptor(returnDict['app'],1,writeData,ffi.sizeof(writeData),ffi.NULL,result_writeData)

results
error code: 
0


In [72]:
@ffi.callback("void(void* , FfiResult*, unsigned long)")
def result_cipher(user_data, result, handle):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(handle)

In [73]:
lib.cipher_opt_new_symmetric(returnDict['app'],ffi.NULL,result_cipher)

results
error code: 
0
2


In [74]:
@ffi.callback("void(void* , FfiResult*, XorNameArray*)")
def result_closeEncryptor(user_data, result,name):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    for i in range(32):
        try:
            print(name[i])
        except:
            pass

In [75]:
lib.idata_close_self_encryptor(returnDict['app'],1,2,ffi.NULL,result_closeEncryptor)

results
error code: 
-16
b'Self-encryption error: Storage error: Routing client error -> Access denied'


In [59]:
lalai={}

In [61]:
@ffi.callback("void(void* , FfiResult* , unsigned long)")
def result_mdata_entries_new(user_data, result, entriesHandle):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    noway = ffi.from_handle(user_data)
    print(entriesHandle)
    print(noway)
    #noway['mdata'] = entriesHandle

lib.mdata_entries_new(returnDict['app'],ffi.new_handle(returnDict),result_mdata_entries_new)

In [ ]:
lib.mdata_entries_new(returnDict['app'],ffi.new_handle(returnDict),result_mdata_entries_new)

results
error code: 
0


In [58]:
@ffi.callback("void(void* , FfiResult*)")
def result_access_container_refresh_access_info(user_data, result):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    #noway['mdata'] = entriesHandle

In [59]:
lib.access_container_refresh_access_info(returnDict['app'],ffi.NULL,result_access_container_refresh_access_info)

results
error code: 
-106
b'Core error: Routing client error -> Requested entry not found'


In [71]:
@ffi.callback("void(void* , FfiResult* , ContainerPermissions*, unsigned long)")
def result_access_container_fetch(user_data, result, permissions, entriesHandle):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    noway = ffi.from_handle(user_data)
    print(entriesHandle)
    print(noway)
    #noway['mdata'] = entriesHandle

In [72]:
lib.access_container_fetch(returnDict['app'],ffi.new_handle(returnDict),result_access_container_fetch)

results
error code: 
-106
b'Core error: Routing client error -> Requested entry not found'


In [115]:
@ffi.callback("void(void* , FfiResult* , AccountInfo*)")
def result_app_account_info(user_data, result, info):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    #returnDict = ffi.from_handle(user_data)
    #if info != ffi.NULL:
    #    returnDict['info'] = safeUtils.copy(info,ffi)
    print(info)

In [116]:
lib.app_account_info(returnDict['app'],ffi.new_handle(returnDict),result_app_account_info)

results
error code: 
-100
b'Core error: Routing client error -> Access denied'
<cdata 'AccountInfo *' NULL>


In [121]:
@ffi.callback("void(void *, FfiResult *, ContainerPermissions *, unsigned long)")
def result_access_container_fetch(user_data, result, permissions, permissionLength):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    #returnDict = ffi.from_handle(user_data)
    if permissions != ffi.NULL:
        returnDict['info'] = safeUtils.copy(permissions,ffi)
    print(permissions[0])

In [122]:
lib.access_container_fetch(returnDict['app'],ffi.new_handle(returnDict),result_access_container_fetch)

results
error code: 
0
<cdata 'ContainerPermissions &' 0x7fbc04015230>


In [127]:
myConPerm=returnDict['info'][0]

In [129]:
ffi.string(myConPerm.cont_name)

b'_music'

In [ ]:
lib.access_container_get_container_mdata_info()

In [62]:
@ffi.callback("void(void *, FfiResult *)")
def result_app_init_logging(user_data, result):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    returnDict = ffi.from_handle(user_data)

In [63]:
output_file_name=ffi.new('char[]',b'/home/yomuga/maidsafe/pySafe/pySafe/log_out')

In [64]:
lib.app_init_logging(output_file_name,ffi.new_handle(returnDict),result_app_init_logging)

results
error code: 
0


In [128]:
@ffi.callback("void(void *, FfiResult *, char *)")
def result_app_output_log_path(user_data, result,logFile):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(ffi.string(logFile))
    returnDict['filePath'] = ffi.string(logFile)

In [131]:
file_name=ffi.new('char[]',b'log_out2')

In [132]:
lib.app_output_log_path(file_name,ffi.new_handle(returnDict),result_app_output_log_path)

results
error code: 
0
b'../compiled_binaries/log_out2'


In [73]:
newMdata=ffi.new('MDataInfo *')

In [74]:
newMdata.type_tag=15017

In [67]:
for i in range(32):
    newMdata.name[i]=i+1

In [68]:
@ffi.callback("void(void *, FfiResult *)")
def result_mdata_put(user_data, result):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))

In [69]:
lib.mdata_put(returnDict['app'],newMdata,0,0,ffi.new_handle(returnDict),result_mdata_put)

results
error code: 
-100
b'Core error: Routing client error -> Access denied'


In [70]:
returnDict['app']

<cdata 'void * *' 0x5642362969c0>

---

In [ ]:
ffi.cdef('''
void app_exe_file_stem(void* user_data, void(*)(void*, FfiResult*, char*));
void login(char* account_locator, char* account_password, void* user_data, void(*)(void*), void(*)(void*, FfiResult*, Authenticator*));
void create_acc(char* account_locator, char* account_password, char* invitation, void* user_data, void(*)(void*), void(*)(void*, FfiResult*, Authenticator*));
''')

In [ ]:
libAuth